<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Generative Adverserial Networks (GANs). In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on Colab or a comparable environment. If something is running longer, doublecheck your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a RNN classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection
* <a href="#p3">Part 3</a>: Describe the difference between a discriminator and generator in a GAN
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [92]:
from tensorflow.keras.datasets import reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [93]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((8982,), (8982,), (2246,), (2246,))

In [94]:
# Demo of encoding

word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.


In [95]:
max([len(word) for word in word_index])

24

In [100]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, SimpleRNN

max_features = 20000
maxlen = 50
batch_size = 64

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [97]:
x_train.shape, x_test.shape

((8982, 500), (2246, 500))

In [98]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, 46)
y_test = np_utils.to_categorical(y_test, 46)

In [102]:
model = Sequential()
model.add(Embedding(max_features, 64))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(64))

model.add(Dense(46, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/10
8982/8982 [==============================] - 26s 3ms/step - loss: 2.2640 - acc: 0.4431 - val_loss: 2.0236 - val_acc: 0.4911
Epoch 2/10
8982/8982 [==============================] - 10s 1ms/step - loss: 1.8861 - acc: 0.5190 - val_loss: 1.8025 - val_acc: 0.5303
Epoch 3/10
8982/8982 [==============================] - 10s 1ms/step - loss: 1.6138 - acc: 0.5828 - val_loss: 1.9079 - val_acc: 0.5223
Epoch 4/10
8982/8982 [==============================] - 10s 1ms/step - loss: 1.3220 - acc: 0.6641 - val_loss: 1.8618 - val_acc: 0.5606
Epoch 5/10
8982/8982 [==============================] - 10s 1ms/step - loss: 0.9901 - acc: 0.7531 - val_loss: 1.9170 - val_acc: 0.5681
Epoch 6/10
8982/8982 [==============================] - 10s 1ms/step - loss: 0.7266 - acc: 0.8198 - val_loss: 2.1229 - val_acc: 0.5539
Epoch 7/10
8982/8982 [==============================] - 10s 1ms/step - loss: 0.5437 - acc: 0.8673 - val_loss: 2.1082 - val_acc: 0.5748
Epoch 8

Conclusion - RNN runs, and gives pretty decent improvement over a naive model. To *really* improve the model, more playing with parameters would help. Also - RNN may well not be the best approach here, but it is at least a valid one.

<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [1]:
!pip install google_images_download

    100% |████████████████████████████████| 911kB 7.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for google-images-download ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": "animal pond", "limit": 5, "print_urls": True}
absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = animal pond
Evaluating...
Starting Download...
Image URL: https://www.enchantedlearning.com/pgifs/Pondanimals.GIF
Completed Image ====> 1.Pondanimals.GIF
Image URL: https://i.ytimg.com/vi/NCbu0TND9vE/hqdefault.jpg
Completed Image ====> 2.hqdefault.jpg
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116_inline.png
Completed Image ====> 3.PKLS4116_inline.png
Image URL: https://get.pxhere.com/photo/water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg
URLError on an image...trying next one... Error: HTTP Error 503: Service Temporarily Unavailable
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116.png
Completed Image ====> 4.PKLS4116.png
Image URL: https://i.pinimg.com/originals/57/5c/5b/575c5b5c441e27ff04eb50571ee30127.jpg
Completed Image ====> 5.575c5b5c441e27ff04eb50571ee30127.jpg

Errors: 1



At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goal* - also check for fish.

In [3]:
import numpy as np

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

Using TensorFlow backend.


In [5]:
def process_img_path(img_path):
    return image.load_img(img_path, target_size=(224, 224))

def img_contains_frog(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    model = ResNet50(weights='imagenet')
    features = model.predict(x)
    results = decode_predictions(features, top=3)[0]
    print(results)
    for entry in results:
        if (entry[1] == 'bullfrog') or (entry[1] == 'tree frog') or (entry[1] == 'tailed frog') or (entry[1] == 'frog'):
            return entry[2]
    return 0.0

In [8]:
img_urls = ["./downloads/animal pond/5.575c5b5c441e27ff04eb50571ee30127.jpg",
            "./downloads/animal pond/4.PKLS4116.png",
           "./downloads/animal pond/3.PKLS4116_inline.png",
           "./downloads/animal pond/2.hqdefault.jpg",
           "./downloads/animal pond/1.Pondanimals.GIF"]

In [9]:
for i in img_urls:
    img_contains_frog(process_img_path(i))

[('n02116738', 'African_hunting_dog', 0.6101571), ('n02105162', 'malinois', 0.19866791), ('n02114712', 'red_wolf', 0.051153056)]
[('n03485794', 'handkerchief', 0.8822726), ('n02834397', 'bib', 0.022680892), ('n03291819', 'envelope', 0.020095171)]
[('n04243546', 'slot', 0.8712449), ('n04476259', 'tray', 0.04993588), ('n03908618', 'pencil_box', 0.023072386)]
[('n01443537', 'goldfish', 0.8495859), ('n01631663', 'eft', 0.06760218), ('n02536864', 'coho', 0.035163548)]
[('n03598930', 'jigsaw_puzzle', 0.8680313), ('n06359193', 'web_site', 0.06410024), ('n02834397', 'bib', 0.021264324)]


* ResNet50 has a hard time evaluating cartoon images.  As seen above, it misclassied every single image that was a cartoon.  This makes sense as the training data is majorly composed of real images, and the network has a hard time adjusting to previously unseen forms of a frog, or a fish (i.e. cartoon forms).  The model instead generalizes the pictures into something it does recognize.. like 'slot', or 'pencil box'.

<a id="p3"></a>
## Part 3 - Generative Adverserial Networks (GANS)

Describe the difference between a discriminator and generator in a GAN in your own words.

__*Your Answer:*__ In a GAN, a generator and discriminator are working towards complete opposite goals.  The discriminator in a GAN, in the case of an image, is trying to determine if an image is real or fake.  The discriminator is able to detect the likelihood of this using some training data that was fed into its network.  So if the Discriminator is trained using a lot of Piccaso paintings, it should be able to discriminate between a fake and real Picasso.  It's goal is to reach 100% accuracy.

The generator on the other hand has the goal to trick the discriminator into thinking that it's outputs are 100% real.  The generator starts off with completely random outputs.  The discriminator naturally is able to tell that the generator's initial outputs are fake.  The trick of the generator is that it then uses the feedback from the discriminator to change the weights of its outputs in order to try and make something that is more likely to pass the test. After enough rejects, the discriminator will want to get to the point that it is generating something that can pass the discriminator test, but is not quite an exact replica of an already existing Picasso.

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?
 * I have the most fun in the predictive modeling part of Data Science.  This along with the story telling aspect of data science I feel is my strongest suit in the field.  I really want to expand my knowledge of best practice when it comes to creating models.
- What area of Data Science would you most like to learn more about, and why?
 * I would like to improve upon my data engineering and pipeline creation skills.  I have the least exposure to these skills, and feel like they are important not only to add to my tools, but to really differentiate myself from others that do not appreciate the entire process of crunching data.
- Where do you think Data Science will be in 5 years?
 * I think data science might become a little more organized or regulated than it is today. There are so many tools to use, most of which are not fully taken advantage of.  I can see how that in itself can halt a lot of organizations from getting started in building their own pipelines and organize their data properly.  With a more regularized framework of what data science should look like, I feel like it will unlock the benefits to many more.
- What are the threats posed by AI to our society?
 * I honestly do not know what the actual threats AI poses to society from what I have learned so far.  Even though it is true that the decisions of a lot of Neural Networks people are building are hazy, I feel like we still understand the limitations of what neural networks can do.  The biggest threat I think is ourselves.  AI, as far as I understand, can only go wrong if the creators are not thoughful about the biases and information they feed into a system.
- How do you think we can counteract those threats? 
 * Having a regularized approach in the creation and research of Neural Networks will be incredibly important.  If there is a uniform way that people are building networks and testing them, just like any other research, others will be able to replicate and confirm findings (or prove wrong). With a uniform framework, udnerstanding can spread faster, and there is less chance of the technology to run away from us.
- Do you think achieving General Artifical Intelligence is ever possible?
 * I do not know enough about current research, or about how general intelligence works in the first place to give a great answer.  But.. I think it is possible. It is not artificial in the way we describe it, but general intelligence has already occured with biology.. naturally.  It took a few billion years, but why would it be harder when there is directed energy and intelligence towards the effort? It may take a few thousand or million of years, but I see no reason that as we continue to learn about the brain, about the universe around us, and improve upon the technology that we currently have at the rate it has been improving.. that at some point we will have the ability to create general intelligence.   



## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [10]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")